# INIT

In [4]:
!python3 -m venv myenvCHAMPI

In [15]:
!python -m ipykernel install --user --name=myenvCHAMPI

Installed kernelspec myenvCHAMPI in /Users/pauldelamarre/Library/Jupyter/kernels/myenvchampi


In [16]:
#Windows
# myenvCHAMPI\Scripts\activate.bat
#Mac
!source myenvCHAMPI/bin/activate

### Relancer la page + selectionner le nouvel env en haut à droite 

In [ ]:
!pip install -r requirements.txt

In [9]:
import numpy as np              # linear algebra
import pandas as pd             # data processing, CSV file I/O (e.g. pd.read_csv)
import os                       # files handling
import re
from random import randint, seed
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

# Téléchargement des données

In [10]:
import json
with open('/Users/pauldelamarre/Desktop/TP_PYTHON/TP_CHAMPI/json_files/observations_11.json') as f:
    data = json.load(f)

In [11]:
df = pd.DataFrame(data)

In [12]:
df.head(5)

,observation,label,image_id,image_url,user,date,gbif_info,thumbnail,location
0,/observer/show_observation/264880,Cortinarius,699257,http://mushroomobserver.org/images/320/699257,/observer/show_user/6796,2016-12-18,"{'status': 'ACCEPTED', 'kingdom': 'Fungi', 'us...",1,/location/show_location/1883
1,/observer/show_observation/264880,Cortinarius,699254,http://mushroomobserver.org/images/320/699254,/observer/show_user/6796,2016-12-18,"{'status': 'ACCEPTED', 'kingdom': 'Fungi', 'us...",0,/location/show_location/1883
2,/observer/show_observation/264880,Cortinarius,699255,http://mushroomobserver.org/images/320/699255,/observer/show_user/6796,2016-12-18,"{'status': 'ACCEPTED', 'kingdom': 'Fungi', 'us...",0,/location/show_location/1883
3,/observer/show_observation/264880,Cortinarius,699256,http://mushroomobserver.org/images/320/699256,/observer/show_user/6796,2016-12-18,"{'status': 'ACCEPTED', 'kingdom': 'Fungi', 'us...",0,/location/show_location/1883
4,/observer/show_observation/264881,Cortinarius variosimilis,699260,http://mushroomobserver.org/images/320/699260,/observer/show_user/120,2016-11-14,"{'kingdom': 'Fungi', 'family': 'Cortinariaceae...",1,/location/show_location/4145


In [13]:
import requests
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import urllib.request
def download_image(url):
    with urllib.request.urlopen(url) as url_response:
        img_data = url_response.read()
    return img_data

In [23]:
res = download_image("https://images.mushroomobserver.org/320/699256.jpg")

In [ ]:
array(res)

# FIN 

In [5]:
!pip freeze > requirements.txt